In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np
from keras.utils import np_utils

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기

# gcc6_0~3
gcc_version = 'gcc3'
gcc6_0_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_0_32.csv", index_col=0)
gcc6_1_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_1_32.csv", index_col=0)
gcc6_2_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_2_32.csv", index_col=0)
gcc6_3_32 = pd.read_csv("binutils_gcc3~9_op0~4_csv/"+gcc_version+"_3_32.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape, gcc6_1_32.shape, gcc6_2_32.shape, gcc6_3_32.shape) # train

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)
gcc6_1_32.reset_index(inplace=True, drop=True)
gcc6_2_32.reset_index(inplace=True, drop=True)
gcc6_3_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
gcc6_0_32.head()

Using TensorFlow backend.


(2129359, 2) (1805387, 2) (1749581, 2) (2076717, 2)
reset_index 완료


,bin,label
0,55,1
1,89,0
2,e5,0
3,83,0
4,ec,0


In [2]:
# (2-1) 데이터체크 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()),len(gcc6_1_32['bin'].unique()))
print(len(gcc6_2_32['bin'].unique()),len(gcc6_3_32['bin'].unique()))

256 256
256 256


In [3]:
# (3) get_dummies 변환 

# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot = pd.get_dummies(gcc6_0_32)
gcc6_1_32_onehot = pd.get_dummies(gcc6_1_32)
gcc6_2_32_onehot = pd.get_dummies(gcc6_2_32)
gcc6_3_32_onehot = pd.get_dummies(gcc6_3_32)
print('원핫인코딩완료')

print(gcc6_0_32_onehot.shape, gcc6_1_32_onehot.shape, gcc6_2_32_onehot.shape, gcc6_3_32_onehot.shape)

원핫인코딩완료
(2129359, 257) (1805387, 257) (1749581, 257) (2076717, 257)


In [4]:
# (4) 데이터 체크 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함

# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32_onehot['label'].value_counts(), '\n', gcc6_1_32_onehot['label'].value_counts())
print(gcc6_2_32_onehot['label'].value_counts(), '\n', gcc6_3_32_onehot['label'].value_counts())

0    2125055
1       4304
Name: label, dtype: int64 
 0    1801378
1       4009
Name: label, dtype: int64
0    1746216
1       3365
Name: label, dtype: int64 
 0    2073836
1       2881
Name: label, dtype: int64


In [5]:
# 전체 행렬에 대한 value_counts()
gcc6_0_32['bin'].value_counts()

00    317465
ff    160053
8b    145301
45    144831
89    102602
       ...  
96       274
a7       272
b3       269
9b       261
a6       237
Name: bin, Length: 256, dtype: int64

In [6]:
# (5-1) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx0 = gcc6_0_32_onehot[gcc6_0_32_onehot['label']==1].index  # 407, 474 ...
ls0 = list(idx0)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx0 = [] 
# left_idx, right_idx = -3, 4  # 3개씩
left_idx, right_idx = -10, 11 # 10개씩
# left_idx, right_idx = -30, 31 # 30개씩
# left_idx, right_idx = -90, 91 # 90개씩


# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx0.extend(list(idx0 + k)) # index 형이라서 가능

ls_idx0 = list(set(ls_idx0))    
# print(len(ls_idx0))

ls_idx0.sort() # 인덱스 정렬
ls_idx0 = list(filter(lambda x: x>0, ls_idx0)) # 0이상인 값들만 
ls_idx0 = list(filter(lambda x: x<len(gcc6_0_32_onehot), ls_idx0))

print('gcc6_0_32', len(ls_idx0))

# loc 로 수정필요
gcc6_0_32_onehot_30gram = gcc6_0_32_onehot.loc[ls_idx0,:].copy()

gcc6_0_32 87541


In [7]:
# (5-2) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx1 = gcc6_1_32_onehot[gcc6_1_32_onehot['label']==1].index  # 407, 474 ...
ls1 = list(idx1)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx1 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx1.extend(list(idx1 + k)) # index 형이라서 가능

ls_idx1 = list(set(ls_idx1))    
# print(len(ls_idx0))

ls_idx1.sort() # 인덱스 정렬
ls_idx1 = list(filter(lambda x: x>0, ls_idx1)) # 0이상인 값들만 
ls_idx1 = list(filter(lambda x: x<len(gcc6_1_32_onehot), ls_idx1))

print('gcc6_1_32', len(ls_idx1))

# loc 로 수정필요
gcc6_1_32_onehot_30gram = gcc6_1_32_onehot.loc[ls_idx1,:].copy()


gcc6_1_32 82130


In [8]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx2 = gcc6_2_32_onehot[gcc6_2_32_onehot['label']==1].index  # 407, 474 ...
ls2 = list(idx2)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx2 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx2.extend(list(idx2 + k)) # index 형이라서 가능

ls_idx2 = list(set(ls_idx2))    
# print(len(ls_idx0))

ls_idx2.sort() # 인덱스 정렬
ls_idx2 = list(filter(lambda x: x>0, ls_idx2)) # 0이상인 값들만 
ls_idx2 = list(filter(lambda x: x<len(gcc6_2_32_onehot), ls_idx2))

print('gcc6_2_32', len(ls_idx2))

# loc 로 수정필요
gcc6_2_32_onehot_30gram = gcc6_2_32_onehot.loc[ls_idx2,:].copy()


gcc6_2_32 67502


In [9]:

# (5-3) gcc6 - 30gram(앞뒤로 3개씩) - 함수의 시작
######################## gcc6_0_32
idx3 = gcc6_3_32_onehot[gcc6_3_32_onehot['label']==1].index  # 407, 474 ...
ls3 = list(idx3)

# value_counts()
# 0    2151901
# 1       4307

# 최종 뽑을 행에 대한 index
ls_idx3 = [] 

# 전체 index에 -10더한 index들 -9 더한 인덱스들, ~~~ 10더한 index들 다 합침.(extend)
for k in range(left_idx, right_idx):
    ls_idx3.extend(list(idx3 + k)) # index 형이라서 가능

ls_idx3 = list(set(ls_idx3))    
# print(len(ls_idx0))

ls_idx3.sort() # 인덱스 정렬
ls_idx3 = list(filter(lambda x: x>0, ls_idx3)) # 0이상인 값들만 
ls_idx3 = list(filter(lambda x: x<len(gcc6_3_32_onehot), ls_idx3))

print('gcc6_3_32', len(ls_idx3))

# loc 로 수정필요
gcc6_3_32_onehot_30gram = gcc6_3_32_onehot.loc[ls_idx3,:].copy()


gcc6_3_32 57769


In [10]:
# (7) 훈련데이터, 훈련라벨 분리 012

# 훈련 데이터, 훈련 라벨
x_gcc6_0_32 = gcc6_0_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_0_32 = gcc6_0_32_onehot_30gram['label'].to_numpy()

x_gcc6_1_32 = gcc6_1_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_1_32 = gcc6_1_32_onehot_30gram['label'].to_numpy()

x_gcc6_2_32 = gcc6_2_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_2_32 = gcc6_2_32_onehot_30gram['label'].to_numpy()

# 테스트 데이터, 테스트 라벨
x_gcc6_3_32 = gcc6_3_32_onehot_30gram.iloc[:,1:].to_numpy()
y_gcc6_3_32 = gcc6_3_32_onehot_30gram['label'].to_numpy()

# train
print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

# test
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(87541, 256) (87541,)
(82130, 256) (82130,)
(67502, 256) (67502,)
(57769, 256) (57769,)


In [11]:
# (8) 훈련데이터, 훈련라벨 reshape 작업 => # ex - (605, 1, 256)

## gcc6_0_32
x_gcc6_0_32 = x_gcc6_0_32.reshape(x_gcc6_0_32.shape[0], 1, x_gcc6_0_32.shape[1])
y_gcc6_0_32 = y_gcc6_0_32.reshape(y_gcc6_0_32.shape[0], 1, 1)

## gcc6_1_32
x_gcc6_1_32 = x_gcc6_1_32.reshape(x_gcc6_1_32.shape[0], 1, x_gcc6_1_32.shape[1])
y_gcc6_1_32 = y_gcc6_1_32.reshape(y_gcc6_1_32.shape[0], 1, 1)

## gcc6_2_32
x_gcc6_2_32 = x_gcc6_2_32.reshape(x_gcc6_2_32.shape[0], 1, x_gcc6_2_32.shape[1])
y_gcc6_2_32 = y_gcc6_2_32.reshape(y_gcc6_2_32.shape[0], 1, 1)

### test data
## gcc6_3_32
x_gcc6_3_32 = x_gcc6_3_32.reshape(x_gcc6_3_32.shape[0], 1, x_gcc6_3_32.shape[1])
y_gcc6_3_32 = y_gcc6_3_32.reshape(y_gcc6_3_32.shape[0], 1, 1)

print(x_gcc6_0_32.shape, y_gcc6_0_32.shape)
print(x_gcc6_1_32.shape, y_gcc6_1_32.shape)
print(x_gcc6_2_32.shape, y_gcc6_2_32.shape)

## test data
print(x_gcc6_3_32.shape, y_gcc6_3_32.shape)

(87541, 1, 256) (87541, 1, 1)
(82130, 1, 256) (82130, 1, 1)
(67502, 1, 256) (67502, 1, 1)
(57769, 1, 256) (57769, 1, 1)


In [12]:
# (9) 최종 훈련데이터, 훈련라벨 concatenate 작업
# 0123 / 3
x_gcc6_0123_32 = np.concatenate((x_gcc6_0_32, x_gcc6_1_32, x_gcc6_2_32, x_gcc6_3_32), axis=0)
y_gcc6_0123_32 = np.concatenate((y_gcc6_0_32, y_gcc6_1_32, y_gcc6_2_32, y_gcc6_3_32), axis=0)
print(x_gcc6_0123_32.shape, y_gcc6_0123_32.shape)

(294942, 1, 256) (294942, 1, 1)


In [13]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 교차검증 kfold
from sklearn.model_selection import KFold

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# K-Fold 객체 생성
kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전

# Confusion Matrix
from sklearn.metrics import confusion_matrix

# ROC Curve
from sklearn.metrics import roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회
for train, validation in kf.split(x_gcc6_0123_32, y_gcc6_0123_32):
    
    xInput = Input(batch_shape=(None, 1, 256)) # 
    xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
    xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

    model1 = Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #print(model1.summary())
    print('======Training stage======')
    model1.fit(x_gcc6_0123_32[train],
               y_gcc6_0123_32[train],
               epochs = 3,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict    
    # predict 값
    k_pr = model1.predict(x_gcc6_0123_32[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0123_32[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
======Training stage======
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/3
265447/265447 [==============================] - 22s 81us/step - loss: 0.0347 - accuracy: 0.9916
Epoch 2/3
265447/265447 [==============================] - 20s 74us/step - loss: 0.0217 - accuracy: 0.9939
Epoch 3/3
265447/265447 [==============================] - 17s 64us/step - loss: 0.0215 - accuracy: 0.9939


accuracy_score 0.9973215799288014
recall_score 1.0
precision_score 0.9482984293193717
f1_score 0.9734632180047027
======Training stage======
Epoch 1/3
265447/265447 [==============================] - 18s 68us/step - loss: 0.0337 - accuracy: 0.9919
Epoch 2/3
265447/265447 [==============================] - 19s 72us/step - loss: 0.0217 - accuracy: 0.9939
Epoch 3/3
265447/265447 [==============================] - 18s 68us/step - loss: 0.0215 - accuracy: 0.9939


accuracy_score 0.9971520596711307
recall_score 1.0
precision_score 0.9455958549222798
f1_score 0.9720372836218376
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 19s 73us/step - loss: 0.0349 - accuracy: 0.9916
Epoch 2/3
265448/265448 [==============================] - 19s 70us/step - loss: 0.0216 - accuracy: 0.9939
Epoch 3/3
265448/265448 [==============================] - 21s 78us/step - loss: 0.0215 - accuracy: 0.9939


accuracy_score 0.9970163423069098
recall_score 1.0
precision_score 0.9424083769633508
f1_score 0.9703504043126685
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 19s 72us/step - loss: 0.0333 - accuracy: 0.9920
Epoch 2/3
265448/265448 [==============================] - 19s 71us/step - loss: 0.0210 - accuracy: 0.9942
Epoch 3/3
265448/265448 [==============================] - 19s 73us/step - loss: 0.0209 - accuracy: 0.9942


accuracy_score 0.9943378314233403
recall_score 1.0
precision_score 0.8957553058676654
f1_score 0.945011524530787
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 20s 75us/step - loss: 0.0343 - accuracy: 0.9918
Epoch 2/3
265448/265448 [==============================] - 20s 73us/step - loss: 0.0214 - accuracy: 0.9940
Epoch 3/3
265448/265448 [==============================] - 21s 79us/step - loss: 0.0212 - accuracy: 0.9940


...

accuracy_score 0.99182884654506
recall_score 0.9965564738292011
precision_score 0.8597742127153892
f1_score 0.9231259968102073
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 20s 77us/step - loss: 0.0331 - accuracy: 0.9921
Epoch 2/3
265448/265448 [==============================] - 20s 75us/step - loss: 0.0200 - accuracy: 0.9943
Epoch 3/3
265448/265448 [==============================] - 21s 78us/step - loss: 0.0198 - accuracy: 0.9943


accuracy_score 0.992913812978911
recall_score 0.9842681258549931
precision_score 0.8855384615384615
f1_score 0.9322967282150955
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 22s 83us/step - loss: 0.0330 - accuracy: 0.9922
Epoch 2/3
265448/265448 [==============================] - 23s 86us/step - loss: 0.0201 - accuracy: 0.9944
Epoch 3/3
265448/265448 [==============================] - 24s 89us/step - loss: 0.0201 - accuracy: 0.9944


accuracy_score 0.9925408557672747
recall_score 0.9932019034670292
precision_score 0.874326750448833
f1_score 0.9299809038828771
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 23s 85us/step - loss: 0.0326 - accuracy: 0.9923
Epoch 2/3
265448/265448 [==============================] - 23s 85us/step - loss: 0.0201 - accuracy: 0.9945
Epoch 3/3
265448/265448 [==============================] - 22s 84us/step - loss: 0.0199 - accuracy: 0.9945


accuracy_score 0.99182884654506
recall_score 0.9945205479452055
precision_score 0.8617210682492582
f1_score 0.9233704292527823
======Training stage======
Epoch 1/3
265448/265448 [==============================] - 24s 91us/step - loss: 0.0326 - accuracy: 0.9924
Epoch 2/3
265448/265448 [==============================] - 24s 89us/step - loss: 0.0198 - accuracy: 0.9946
Epoch 3/3
265448/265448 [==============================] - 24s 89us/step - loss: 0.0197 - accuracy: 0.9946


accuracy_score 0.9909473113175561
recall_score 0.9918864097363083
precision_score 0.8519163763066202
f1_score 0.9165885660731021

K-fold cross validation Accuracy: [0.9973215799288014, 0.9971520596711307, 0.9970163423069098, 0.9943378314233403, 0.9961687122804638, 0.99182884654506, 0.992913812978911, 0.9925408557672747, 0.99182884654506, 0.9909473113175561]

K-fold cross validation Recall: [1.0, 1.0, 1.0, 1.0, 1.0, 0.9965564738292011, 0.9842681258549931, 0.9932019034670292, 0.9945205479452055, 0.9918864097363083]

K-fold cross validation Precision: [0.9482984293193717, 0.9455958549222798, 0.9424083769633508, 0.8957553058676654, 0.9275641025641026, 0.8597742127153892, 0.8855384615384615, 0.874326750448833, 0.8617210682492582, 0.8519163763066202]

K-fold cross validation F1-Score: [0.9734632180047027, 0.9720372836218376, 0.9703504043126685, 0.945011524530787, 0.9624210176255404, 0.9231259968102073, 0.9322967282150955, 0.9299809038828771, 0.9233704292527823, 0.9165885660731021]


In [14]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9942056198764508
10-Fold Cross_validation. Recall : 0.9960433460832737
10-Fold Cross_validation. Precision : 0.8992898938895333
10-Fold Cross_validation. F1-Score : 0.9448646072329601
